In [1]:
import os
import sys
import requests
import time
import json
import getpass

## Input user credentials and configuration

In [ ]:
username = getpass.getpass()

In [ ]:
password = getpass.getpass()

In [7]:
base_domain = 'des.ncsa.illinois.edu'
config = {
    'auth_token': '',
    'apiBaseUrl': 'https://{}/desaccess/api'.format(base_domain),
    'filesBaseUrl': 'https://{}/files-desaccess'.format(base_domain),
    'username': username,
    'password': password,
    'database': 'desdr',
    'release': 'dr2',
}

## Example positions table for cutout requests

In [8]:
positions = '''
RA,DEC,COADD_OBJECT_ID,XSIZE,YSIZE,COLORS_FITS,RGB_STIFF_COLORS,RGB_LUPTON_COLORS,RGB_MINIMUM,RGB_STRETCH,RGB_ASINH,MAKE_FITS,MAKE_RGB_STIFF,MAKE_RGB_LUPTON
46.275669001230,-34.256000012300,,0.90,1.30,,,,1.0,50.0,10.0,true,false,false
36.608400001230,-15.688890012300,,0.90,1.30,z,grz,,1.0,50.0,10.0,false,false,true
21.588130000000,3.486110000000,,0.70,1.10,grz,riy,riy,1.0,50.0,10.0,true,true,true
'''

## Define API wrapper functions

### Define login function

In [9]:
def login():
    """Obtains an auth token using the username and password credentials for a given database.
    """
    # Login to obtain an auth token
    r = requests.post(
        '{}/login'.format(config['apiBaseUrl']),
        data={
            'username': config['username'],
            'password': config['password'],
            'database': config['database']
        }
    )
    # Store the JWT auth token
    config['auth_token'] = r.json()['token']
    return config['auth_token']


### Define cutout job submission function

In [10]:
def submit_cutout_job(data = {
        'db': config['database'],
        'release': config['release']
    }):
    """Submits a query job and returns the complete server response which includes the job ID."""

    # Submit job
    r = requests.put(
        '{}/job/cutout'.format(config['apiBaseUrl']),
        data=data,
        headers={'Authorization': 'Bearer {}'.format(config['auth_token'])}
    )
    response = r.json()
    # print(json.dumps(response, indent=2))
    
    if response['status'] == 'ok':
        job_id = response['jobid']
        print('Job "{}" submitted.'.format(job_id))
        # Refresh auth token
        config['auth_token'] = response['new_token']
    else:
        job_id = None
        print('Error submitting job: '.format(response['message']))
    
    return response


### Define query job submission function

In [19]:
def submit_query_job():
    """Submits a query job and returns the complete server response which includes the job ID."""

    # Specify API request parameters
    data = {
        'username': config['username'],
        'db': config['database'],
        'filename': 'results.csv',
        'query': '''
            SELECT
            COADD_OBJECT_ID,RA,DEC,
            MAG_AUTO_G G,
            MAG_AUTO_R R,
            WAVG_MAG_PSF_G G_PSF,
            WAVG_MAG_PSF_R R_PSF
            FROM DR2_MAIN
            WHERE
            RA between 323.36-0.12 and 323.36+0.12 and
            DEC between -0.82-0.12 and -0.82+0.12 and
            WAVG_SPREAD_MODEL_I + 3.0*WAVG_SPREADERR_MODEL_I < 0.005 and
            WAVG_SPREAD_MODEL_I > -1 and
            IMAFLAGS_ISO_G = 0 and
            IMAFLAGS_ISO_R = 0 and
            FLAGS_G < 4 and
            FLAGS_R < 4
        '''
    }

    # Submit job
    r = requests.put(
        '{}/job/query'.format(config['apiBaseUrl']),
        data=data,
        headers={'Authorization': 'Bearer {}'.format(config['auth_token'])}
    )
    response = r.json()
    
    if response['status'] == 'ok':
        job_id = response['jobid']
        print('Job "{}" submitted.'.format(job_id))
        # Refresh auth token
        config['auth_token'] = response['new_token']
    else:
        job_id = None
        print('Error submitting job: '.format(response['message']))
    
    return response

### Define job status function

In [12]:
def get_job_status(job_id):
    """Returns the current status of the job identified by the unique job_id."""

    r = requests.post(
        '{}/job/status'.format(config['apiBaseUrl']),
        data={
            'job-id': job_id
        },
        headers={'Authorization': 'Bearer {}'.format(config['auth_token'])}
    )
    response = r.json()
    # Refresh auth token
    config['auth_token'] = response['new_token']
    # print(json.dumps(response, indent=2))
    return response

### Define job file downloader function

In [13]:
def download_job_files(url, outdir):
    os.makedirs(outdir, exist_ok=True)
    r = requests.get('{}/json'.format(url))
    for item in r.json():
        if item['type'] == 'directory':
            suburl = '{}/{}'.format(url, item['name'])
            subdir = '{}/{}'.format(outdir, item['name'])
            download_job_files(suburl, subdir)
        elif item['type'] == 'file':
            data = requests.get('{}/{}'.format(url, item['name']))
            with open('{}/{}'.format(outdir, item['name']), "wb") as file:
                file.write(data.content)

    response = r.json()
    return response

### Define job file lister function

In [14]:
def list_job_files(url):
    r = requests.get('{}/json'.format(url))
    for item in r.json():
        if item['type'] == 'directory':
            suburl = '{}/{}'.format(url, item['name'])
            subdir = '{}/{}'.format(outdir, item['name'])
            list_job_files(suburl, subdir)
        elif item['type'] == 'file':
            print('{}/{}'.format(url, item['name']))
    response = r.json()
    return response

### Define downloaded file lister

In [15]:
def list_downloaded_files(download_dir):
    for dirpath, dirnames, filenames in os.walk(download_dir):
        for filename in filenames:
            print(os.path.join(dirpath, filename))


### Define job status polling function

In [16]:
def job_status_poll(job_id):
    print('Polling status of job "{}"...'.format(job_id), end='')
    job_status = ''
    response = None
    while job_status != 'ok':
        # Fetch the current job status
        response = get_job_status(job_id)
        # Quit polling if there is an error getting a status update
        if response['status'] != 'ok':
            break
        job_status = response['jobs'][0]['job_status']
        if job_status == 'success' or job_status == 'failure':
            print('\nJob completed with status: {}'.format(job_status))
            break
        else:
            # Display another dot to indicate that polling is still active
            print('.', end='', sep='', flush=True)
        time.sleep(3)
    return response

## Obtain authentication token

In [ ]:
 # Authenticate and store the auth token for subsequent API calls
try:
    print('Logging in as user "{}" ("{}") and storing auth token...'.format(config['username'], config['database']))
    login()
except:
    print('Login failed.')
    sys.exit(1)

## Submit jobs

### Submit a query job to retrieve data from the DESSCI database

In [20]:
job_type = 'query'

print('Submitting query job...')
response = submit_query_job()

# Store the unique job ID for the new job
job_id = response['jobid']
print('New job submitted: {}'.format(job_id))

response = job_status_poll(job_id)

Submitting query job...
Job "dd0d68c481fb46fba05897e52579392a" submitted.
New job submitted: dd0d68c481fb46fba05897e52579392a
Polling status of job "dd0d68c481fb46fba05897e52579392a"..........
Job completed with status: success


### Download the job output files

In [24]:
# If successful, display job results
if response['status'] == 'ok':
    job_type = response['jobs'][0]['job_type']
    job_id = response['jobs'][0]['job_id']
    # Construct the job file download URL
    job_url = '{}/{}/{}/{}'.format(config['filesBaseUrl'], config['username'], job_type, job_id)
    download_dir = './{}'.format(job_id)
    # Download each raw job file sequentially to a subfolder of the working directory
    download_job_files(job_url, download_dir)
    print('Files for job "{}" downloaded to "{}"'.format(job_id, download_dir))
    list_downloaded_files(download_dir)
else:
    print('The job "{}" failed.'.format(job_id))

Files for job "dd0d68c481fb46fba05897e52579392a" downloaded to "./dd0d68c481fb46fba05897e52579392a"
./dd0d68c481fb46fba05897e52579392a/meta.json
./dd0d68c481fb46fba05897e52579392a/results.csv


### Submit a cutout job to retrieve image data from the DESDR database

In [ ]:
job_type = 'cutout'
data = {
        'db': config['database'],
        'release': config['release'],
        'positions': positions,
    }
print('Submitting cutout job...')
response = submit_cutout_job(data=data)

if response['status'] == 'ok':
    # Store the unique job ID for the new job
    job_id = response['jobid']
    print('New job submitted: {}'.format(job_id))
    response = job_status_poll(job_id)
    print(json.dumps(response, indent=2))
else:
    print('Response: {}'.format(json.dumps(response, indent=2)))


### Cutout request that discards unrequested FITS files

In [ ]:
job_type = 'cutout'

data = {
        'db': config['database'],
        'release': config['release'],
        'positions': positions,
        'discard_fits_files': True,
    }
print('Submitting cutout job...')
response = submit_cutout_job(data=data)

if response['status'] == 'ok':
    # Store the unique job ID for the new job
    job_id = response['jobid']
    print('New job submitted: {}'.format(job_id))
    response = job_status_poll(job_id)
    print(json.dumps(response, indent=2))
else:
    print('Response: {}'.format(json.dumps(response, indent=2)))


### Download the job output files

In [27]:
# If successful, display job results
if response['status'] == 'ok':
    job_type = response['jobs'][0]['job_type']
    job_id = response['jobs'][0]['job_id']
    # Construct the job file download URL
    job_url = '{}/{}/{}/{}'.format(config['filesBaseUrl'], config['username'], job_type, job_id)
    download_dir = './{}'.format(job_id)
    # Download each raw job file sequentially to a subfolder of the working directory
    download_job_files(job_url, download_dir)
    print('Files for job "{}" downloaded to "{}"'.format(job_id, download_dir))
    list_downloaded_files(download_dir)
else:
    print('The job "{}" failed.'.format(job_id))

Files for job "ec10fde881944294b08cf214f7896794" downloaded to "./ec10fde881944294b08cf214f7896794"
./ec10fde881944294b08cf214f7896794/positions_ec10fde881944294b08cf214f7896794.csv
./ec10fde881944294b08cf214f7896794/summary.json
./ec10fde881944294b08cf214f7896794/cutout_ec10fde881944294b08cf214f7896794.log
./ec10fde881944294b08cf214f7896794/DES0226-1541/DESJ022626.0160-154120.0040/DESJ022626.0160-154120.0040_gri_lupton.png
./ec10fde881944294b08cf214f7896794/DES0305-3415/DESJ030506.1606-341521.6000/DESJ030506.1606-341521.6000_i.fits
./ec10fde881944294b08cf214f7896794/DES0126+0335/DESJ012621.1512+032909.9960/DESJ012621.1512+032909.9960_g.fits
./ec10fde881944294b08cf214f7896794/DES0126+0335/DESJ012621.1512+032909.9960/DESJ012621.1512+032909.9960_riy_stiff.png
./ec10fde881944294b08cf214f7896794/DES0126+0335/DESJ012621.1512+032909.9960/DESJ012621.1512+032909.9960_riy_lupton.png
./ec10fde881944294b08cf214f7896794/DES0126+0335/DESJ012621.1512+032909.9960/DESJ012621.1512+032909.9960_r.fits
./